## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merge_df = pd.merge(mouse_metadata, study_results, on="Mouse ID")

# Display the data table for preview
merge_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [2]:
# Checking the number of mice.
count_ID = merge_df["Mouse ID"].unique()
len(count_ID)

249

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
merge_df['unique_id'] = merge_df['Mouse ID'].astype(str) + '_' + merge_df['Timepoint'].astype(str)
count = merge_df["unique_id"].value_counts()

# If there are any values that show up more than once they are duplicates
count.head(10)

g989_10    2
g989_5     2
g989_15    2
g989_20    2
g989_0     2
k894_25    1
k403_10    1
k804_20    1
z578_15    1
b559_5     1
Name: unique_id, dtype: int64

In [ ]:
# Mouse ID "g989" has duplicates and needs to be removed from the data.

In [4]:
# Optional: Get all the data for the duplicate mouse ID. 
DupMouse_df = merge_df.loc[merge_df["Mouse ID"] == "g989",:]
DupMouse_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,unique_id
908,g989,Propriva,Female,21,26,0,45.000000,0,g989_0
909,g989,Propriva,Female,21,26,0,45.000000,0,g989_0
910,g989,Propriva,Female,21,26,5,48.786801,0,g989_5
911,g989,Propriva,Female,21,26,5,47.570392,0,g989_5
912,g989,Propriva,Female,21,26,10,51.745156,0,g989_10
913,g989,Propriva,Female,21,26,10,49.880528,0,g989_10
914,g989,Propriva,Female,21,26,15,51.325852,1,g989_15
915,g989,Propriva,Female,21,26,15,53.442020,0,g989_15
916,g989,Propriva,Female,21,26,20,55.326122,1,g989_20
917,g989,Propriva,Female,21,26,20,54.657650,1,g989_20


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
allColumns = merge_df.columns
CleanedMouse_df = merge_df.loc[merge_df["Mouse ID"] != "g989", allColumns]
CleanedMouse_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,unique_id
0,k403,Ramicane,Male,21,16,0,45.000000,0,k403_0
1,k403,Ramicane,Male,21,16,5,38.825898,0,k403_5
2,k403,Ramicane,Male,21,16,10,35.014271,1,k403_10
3,k403,Ramicane,Male,21,16,15,34.223992,1,k403_15
4,k403,Ramicane,Male,21,16,20,32.997729,1,k403_20
...,...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2,z969_25
1889,z969,Naftisol,Male,9,30,30,65.841013,3,z969_30
1890,z969,Naftisol,Male,9,30,35,69.176246,4,z969_35
1891,z969,Naftisol,Male,9,30,40,70.314904,4,z969_40


In [6]:
#Checking that there are no longer duplicates
cleancount = CleanedMouse_df["unique_id"].value_counts()
cleancount.head(10)

x402_45    1
c264_30    1
k894_25    1
k403_10    1
k804_20    1
z578_15    1
b559_5     1
y478_20    1
f129_10    1
t565_40    1
Name: unique_id, dtype: int64

In [7]:
# Checking the number of mice in the clean DataFrame.
cleancount_ID = CleanedMouse_df["Mouse ID"].unique()
len(cleancount_ID)

248

## Summary Statistics

In [17]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

Ramicane_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Ramicane", :]
Capomulin_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Capomulin", :]
Infubinol_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Infubinol", :]
Placebo_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Placebo", :]
Ceftamin_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Ceftamin", :]
Stelasyn_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Stelasyn", :]
Ketapril_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Ketapril", :]
Propriva_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Propriva", :]
Naftisol_df = CleanedMouse_df.loc[CleanedMouse_df["Drug Regimen"] == "Naftisol", :]

Ramicanemean = Ramicane_df["Tumor Volume (mm3)"].mean()
Capomulinmean = Capomulin_df["Tumor Volume (mm3)"].mean()
Infubinolmean = Infubinol_df["Tumor Volume (mm3)"].mean()
Placebomean = Placebo_df["Tumor Volume (mm3)"].mean()
Ceftaminmean1 = Ramicane_df["Tumor Volume (mm3)"].mean()
Stelasynmean1 = Ramicane_df["Tumor Volume (mm3)"].mean()
Ketaprilmean1 = Ramicane_df["Tumor Volume (mm3)"].mean()
Propriva1 = Ramicane_df["Tumor Volume (mm3)"].mean()
Naftisol1 = Ramicane_df["Tumor Volume (mm3)"].mean()
mean1 = Ramicane_df["Tumor Volume (mm3)"].mean()

40.2167450667105

In [14]:
CleanedMouse_df["Drug Regimen"].unique()

array(['Ramicane', 'Capomulin', 'Infubinol', 'Placebo', 'Ceftamin',
       'Stelasyn', 'Zoniferol', 'Ketapril', 'Propriva', 'Naftisol'],
      dtype=object)

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
